# 数据获取

## 船用气象仪数据

In [ ]:
import pandas as pd 
def read_routell(date):
    import warnings;warnings.filterwarnings('ignore')
    ll_dir = 'E:/programming/code_2022_5/route/经纬度走行气象数据/'
    df = pd.read_table(ll_dir+date+'.TXT',sep=',',header=None,usecols=range(0,30))
    pd.set_option('display.max_columns', None)
    df.columns=['output_mark','yymmddhhmmss','0W1Bj','lat','SN','lon','EW',
                'speed_GPS','orient_GPS','GPS_situa',
                'speed_zhenFeng','speed_source','speed_valid',
                'orient_zhenFeng','orient_source','orient_valid',
               'speed_relative','orient_relative','speed_true','orient_true',
               'speed_true_1min','orient_true_1min','speed_true_10min','orient_true_10min',
               'press','press_1min','temp','temp_1min','rh','rh_1min']
    return df
df = read_routell('20220410')
df=df.append(read_routell('20220411'))
df=df.append(read_routell('20220412'))
df=df.append(read_routell('20220413'))
df=df.append(read_routell('20220414'))
df=df.append(read_routell('20220415'))
df=df.append(read_routell('20220416'))
df=df.append(read_routell('20220417'))
df=df.append(read_routell('20220418'))
df=df.append(read_routell('20220419'))
df=df.append(read_routell('20220420'))
df=df.append(read_routell('20220421'))
df=df.append(read_routell('20220422'))
df=df.append(read_routell('20220423'))
df=df.append(read_routell('20220424'))
df=df.append(read_routell('20220425'))
df=df.append(read_routell('20220426'))
# pd.set_option('display.max_rows', None)    #显示所有行
df = df.reset_index()
display(df)

## PDF-站位数据

In [ ]:
def read_spots(dir_name):
    import pandas as pd 
    import datetime
    import warnings;warnings.filterwarnings('ignore')
    df = pd.read_excel(dir_name)
    df.columns=['index','spot_name','lon','lat','depth']
    display(df)
    return df
spots = read_spots('E:/programming/code_2022_5/route/spots/spots_info.xlsx')
# 剔除本次航程未抵达的站位
spots = spots[(spots.spot_name != 'YE-1') & (spots.spot_name != 'YE-2')]
display(spots)

# 数据清洗

In [ ]:
# 去除经纬度异常值
df = df[df['lon']!='XXXXX.XXXX']
df = df.reset_index()
display(df)

# 数据提取

In [ ]:
import numpy as np 
lon=df['lon'].astype('float')//100+df['lon'].astype('float')/100%1*100/60
lon=np.array(lon)
lat=df['lat'].astype('float')//100+df['lat'].astype('float')/100%1*100/60
lat=np.array(lat)
display(lon,lat)

In [ ]:
from cartopy.io.shapereader import Reader
import cartopy.crs as ccrs 
shp_dir='E:/official_shapefiles/'
shp_name='continents_lines.shp'
proj=ccrs.PlateCarree()

In [ ]:
def set_geo(ax):
    import cartopy.feature as cf
    shp = cf.ShapelyFeature(Reader(shp_dir+shp_name).geometries(), ccrs.PlateCarree())
    # proj=ccrs.LambertConformal()
    extent=[117,130,24, 38]
    ax.set_extent(extent, crs=ccrs.PlateCarree()) 
#     ax.add_feature(cf.OCEAN, facecolor='#97DBF2')
#     ax.patch.set_facecolor('#97DBF2')
#     ax.add_feature(cf.LAND)
    ax.add_feature(shp, linestyle=(0,(5,0)), linewidth=0.5, 
        edgecolor='None', facecolor='#bfbfbf',alpha=1,zorder=4)
    #------------------------------------------------------------------------------------------------------------------------------------
#     lb=ax.gridlines(draw_labels=None, xlocs=range(122,126,1), ylocs=range(25,33,1), linestyle=(0,(10,10)),
#                     linewidth=0.25, color='dimgrey', alpha=0.8, zorder=4)
    lb=ax.gridlines(draw_labels=True,x_inline=False, y_inline=False,xlocs=range(118,130,2), ylocs=range(26,38,2),
                    linewidth=0.1, color='gray', alpha=0.8, linestyle='--',zorder=11)
    lb.top_labels = None
    lb.right_labels = None
    lb.rotate_labels = False
    #----------------------------------------------------------
    

In [ ]:
def add_baihua(ax):
    shp=Reader(shp_dir+shp_name)
    for i,s in enumerate(shp.records()):
        ax.text((s.bounds[0]+s.bounds[2])/2,
            (s.bounds[1]+s.bounds[3])/2,
            '{}'.format(i))
        ax.add_geometries([s.geometry],
                      proj,facecolor='none',
                      edgecolor='k',alpha=0.75)
def plot_spots():
    import matplotlib.pyplot as plt
    plt.rcParams['axes.unicode_minus'] = False 
    plt.rcParams['font.sans-serif'] = ['Times New Roman']
    fig = plt.figure(figsize=(8, 8))
    ax = plt.axes(projection=proj)
    set_geo(ax)
    
    #提取数据
    plt.scatter(spots.lon,spots.lat,s=0.2,c='black')
    plt.savefig('spots_20220528.png',dpi=300,bbox_inches='tight',pad_inches=0)

# plot_spots()

In [ ]:
def plot_route():
    import matplotlib.pyplot as plt
    import datetime
    plt.rcParams['axes.unicode_minus'] = False 
    plt.rcParams['font.sans-serif'] = ['Times New Roman']
    fig = plt.figure(figsize=(8, 8))
    ax = plt.axes(projection=proj)
    set_geo(ax)
    #船用气象仪
    ax.plot(lon[:14700],lat[:14700],linewidth=0.5,color='black',zorder=1,label='Go')
    ax.plot(lon[14700:],lat[14700:],'--',linewidth=1,color='black',zorder=1,label='Return')
    #站位信息
    ax.scatter(spots.lon,spots.lat,s=2.5,c='red',zorder=10,label='CTD Station')
    
    
    plt.legend(loc='lower right')
    today = datetime.strftime(datetime.now(),'%Y%m%d')
    plt.savefig('route_{}.png'.format(today),dpi=600,bbox_inches='tight',pad_inches=0)
plot_route()

# 读取Hysplit后向轨迹数据

In [ ]:
def read_hysplit(date):
    import warnings;warnings.filterwarnings('ignore')
    dir = 'E:/programming/code_2022_5/Hysplit/output/'
    df = pd.read_csv(dir+date,sep='\s+',header=None,skiprows=13)
    pd.set_option('display.max_columns', None)
    df.columns=['traj_num','meteo_grid_num','year','month','day','hour','minute',
                'forecast_hour','age_of_traj','lat','lon','h_agl','diagn_var']
    # display(df)
    return df
hysplit=read_hysplit('20220410').groupby("traj_num")
siz=len(hysplit)


In [ ]:
def plot_routewithbackward():
    import matplotlib.pyplot as plt
    plt.rcParams['axes.unicode_minus'] = False 
    plt.rcParams['font.sans-serif'] = ['Times New Roman']
    fig = plt.figure(figsize=(8, 8))
    ax = plt.axes(projection=proj)
    set_geo(ax)
    #船用气象仪
    ax.plot(lon[:14700],lat[:14700],linewidth=0.5,color='black',zorder=6,label='Go')
    ax.plot(lon[14700:],lat[14700:],'--',linewidth=1,color='black',zorder=6,label='Return')
    #站位信息
    ax.scatter(spots.lon,spots.lat,s=2.5,c='red',zorder=10,label='CTD Station')
    #后向轨迹
    for i in range(len(hysplit)):
        ax.plot(hysplit.get_group(i+1).lon,hysplit.get_group(i+1).lat,linewidth=0.5,label='Hysplit_'+str(i+1),zorder=5)
    
    plt.legend(loc='lower right')
    plt.savefig('routewithbackward_20220609.png',dpi=600,bbox_inches='tight',pad_inches=0)
plot_routewithbackward()